In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os

In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return ' '.join([i.strip() for i in filter(None, text.split())])

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))
    
min_line_length = 2
max_line_length = 5
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

question_test = short_questions[-500:]
answer_test = short_answers[-500:]
short_questions = short_questions[:-500]
short_answers = short_answers[:-500]

In [4]:
concat_from = ' '.join(short_questions+question_test).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 8339
Most common words [('you', 4774), ('i', 3186), ('is', 2970), ('what', 1952), ('it', 1904), ('that', 1766)]
Sample data [7, 42, 396, 37, 50, 46, 13, 18, 78, 433] ['what', 'good', 'stuff', 'she', 'okay', 'they', 'do', 'to', 'hey', 'sweet']
filtered vocab size: 8343
% of vocab used: 100.05%


In [5]:
concat_to = ' '.join(short_answers+answer_test).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab from size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])
print('filtered vocab size:',len(dictionary_to))
print("% of vocab used: {}%".format(round(len(dictionary_to)/vocabulary_size_to,4)*100))

vocab from size: 8170
Most common words [('you', 4029), ('i', 3618), ('is', 2533), ('it', 2007), ('not', 1822), ('what', 1548)]
Sample data [10, 219, 4, 5, 341, 42, 50, 12, 8, 123] ['the', 'real', 'you', 'i', 'hope', 'so', 'they', 'do', 'not', 'hi']
filtered vocab size: 8174
% of vocab used: 100.05%


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
for i in range(len(short_answers)):
    short_answers[i] += ' EOS'

In [8]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, batch_size,
                 grad_clip=5.0, beam_width=5, force_teaching_ratio=0.5):
        
        def cells(size, reuse=False):
            return tf.contrib.rnn.DropoutWrapper(
                tf.nn.rnn_cell.LSTMCell(size, initializer=tf.orthogonal_initializer(),reuse=reuse),
                input_keep_prob=0.8,
                output_keep_prob=0.8,
                state_keep_prob=0.8)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.placeholder(tf.int32, [None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        def bahdanau(size):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size, 
                                                                    memory = encoder_embedded)
            return tf.contrib.seq2seq.AttentionWrapper(cell = cells(size), 
                                                        attention_mechanism = attention_mechanism,
                                                        attention_layer_size = size)
        
        def luong(size):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size, 
                                                                    memory = encoder_embedded)
            return tf.contrib.seq2seq.AttentionWrapper(cell = cells(size), 
                                                        attention_mechanism = attention_mechanism,
                                                        attention_layer_size = size)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = bahdanau(size_layer//2),
                cell_bw = luong(size_layer//2),
                inputs = encoder_embedded,
                sequence_length = self.X_seq_len,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
            
        bi_state_c = tf.concat((state_fw[0].c, state_bw[0].c), -1)
        bi_state_h = tf.concat((state_fw[0].h, state_bw[0].h), -1)
        bi_lstm_state = tf.nn.rnn_cell.LSTMStateTuple(c=bi_state_c, h=bi_state_h)
        encoder_state = tuple([bi_lstm_state] * num_layers)
        
        dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units = size_layer, 
                memory = encoder_embedded,
                memory_sequence_length = self.X_seq_len)
            luong_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_embedded,
                memory_sequence_length = self.X_seq_len)
            bahdanau_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            decoder_cells = tf.nn.rnn_cell.MultiRNNCell([luong_cells, bahdanau_cells])
            main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
            decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
            training_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
            inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.Y_seq_len,
                embedding = decoder_embeddings,
                sampling_probability = 1 - force_teaching_ratio,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = training_helper,
                initial_state = decoder_cells.zero_state(batch_size, tf.float32),
                output_layer = tf.layers.Dense(to_dict_size))
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_embedded, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            luong_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer,reuse=True) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            bahdanau_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer,reuse=True) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            decoder_cells = tf.nn.rnn_cell.MultiRNNCell([luong_cells, bahdanau_cells])
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cells,
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cells.zero_state(batch_size * beam_width, tf.float32),
                beam_width = beam_width,
                output_layer = tf.layers.Dense(to_dict_size, _reuse=True),
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.predicting_ids = predicting_decoder_output.predicted_ids[:, :, 0]
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.logits,
                                                     targets = self.Y,
                                                     weights = masks)
        l2 = sum(1e-5 * tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
        self.cost += l2
        params = tf.trainable_variables()
        gradients = tf.gradients(self.cost, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, grad_clip)
        self.optimizer = tf.train.AdamOptimizer().apply_gradients(zip(clipped_gradients, params))

In [9]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-2
batch_size = 16
epoch = 30

In [10]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), batch_size, learning_rate)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            try:
                ints.append(dic[k])
            except Exception as e:
                print(e)
                ints.append(UNK)
        X.append(ints)
    return X

In [12]:
X = str_idx(short_questions, dictionary_from)
Y = str_idx(short_answers, dictionary_to)
X_test = str_idx(question_test, dictionary_from)
Y_test = str_idx(answer_test, dictionary_from)

'touchdown'
"president's"
'ucleanu'
'albanian'
'campaign'
'givem'
'wantbut'
'huhwhy'
"some'b'y"
'guards'
'surest'
'leopard'
'kittens'
'compassion'
'pond'
'anacott'
'unloading'
'faa'
'allright'
'jenkins'
'vell'
'boththought'
'wives'
'kambucha'
'mushroom'
'adamant'
'spencer'
'justwhispering'
'isnot'
'relief'
'heartbeats'
'hel1'
'lamplight'
'bp'
'captaings'
'sut'
'mckinley'
'forged'
'dentist'
'dentist'
'promoted'
'whiff'
'reinvented'
"prom's"
'knowtake'
'formality'
'biologically'
'tripledogdare'
'concern'
'supreme'
'sovereign'
'sexuality'
'benedict'
'ryan'
'antigua'
"lombardo's"
'settlement'
'jumper'
'jumping'
'kicking'
'board'
'adjusted'
'richards'
'smelting'
'retort'
'sorsha'
"bavmorda's"
'douse'
'warrior'
"goat's"
'limitless'
'courage'
'amish'
'elsesam'
'hochmut'
'1790'
'amish'
'fiftyseven'
'morally'
"ethic'ly"
'hurtcha'
'ohh'
'ohh'
'ohh'
'yesthe'
'dor'
'gulch'
'ohhhh'
'davidov'
'couldnt'
'splitting'
'elektra'
'elektra'
'wouldnt'
'maybethe'
'youll'
'shaken'
'stirred'
'bondjamesbond'
'c

In [13]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

def check_accuracy(logits, Y):
    acc = 0
    for i in range(logits.shape[0]):
        internal_acc = 0
        count = 0
        for k in range(len(Y[i])):
            try:
                if Y[i][k] == logits[i][k]:
                    internal_acc += 1
                count += 1
                if Y[i][k] == EOS:
                    break
            except:
                break
        acc += (internal_acc / count)
    return acc / logits.shape[0]

In [14]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    X, Y = shuffle(X, Y)
    for k in range(0, (len(short_questions) // batch_size) * batch_size, batch_size):
        batch_x, seq_x = pad_sentence_batch(X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: k+batch_size], PAD)
        predicted, loss, _ = sess.run([model.predicting_ids, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y,
                                                model.X_seq_len:seq_x,
                                                model.Y_seq_len:seq_y})
        print(loss)
        total_loss += loss
        total_accuracy += check_accuracy(predicted,batch_y)
    total_loss /= (len(short_questions) // batch_size)
    total_accuracy /= (len(short_answers) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 8.179269, avg accuracy: 0.177249
epoch: 2, avg loss: 5.603351, avg accuracy: 0.190242
epoch: 3, avg loss: 5.359934, avg accuracy: 0.221061
epoch: 4, avg loss: 5.138546, avg accuracy: 0.242512
epoch: 5, avg loss: 4.874407, avg accuracy: 0.254047
epoch: 6, avg loss: 4.564264, avg accuracy: 0.273051
epoch: 7, avg loss: 4.235640, avg accuracy: 0.300116
epoch: 8, avg loss: 3.971333, avg accuracy: 0.323290
epoch: 9, avg loss: 3.798956, avg accuracy: 0.341910
epoch: 10, avg loss: 3.698873, avg accuracy: 0.353596
epoch: 11, avg loss: 3.607705, avg accuracy: 0.367085
epoch: 12, avg loss: 3.525176, avg accuracy: 0.377526
epoch: 13, avg loss: 3.450969, avg accuracy: 0.388047
epoch: 14, avg loss: 3.391227, avg accuracy: 0.399133
epoch: 15, avg loss: 3.324066, avg accuracy: 0.409852
epoch: 16, avg loss: 3.278514, avg accuracy: 0.419182
epoch: 17, avg loss: 3.227930, avg accuracy: 0.429459
epoch: 18, avg loss: 3.188380, avg accuracy: 0.437290
epoch: 19, avg loss: 3.146161, avg ac

In [15]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: the door was unlocked
REAL ANSWER: lock's busted
PREDICTED ANSWER: in busted 

row 2
QUESTION: that is this
REAL ANSWER: my apartment
PREDICTED ANSWER: that is 

row 3
QUESTION: you alright
REAL ANSWER: let's just find someplace
PREDICTED ANSWER: yeah i 

row 4
QUESTION: yes a little girl
REAL ANSWER: where is she
PREDICTED ANSWER: where is 

row 5
QUESTION: she lived there
REAL ANSWER: i do not mean her
PREDICTED ANSWER: i do not know her 

row 6
QUESTION: goodnight jeff
REAL ANSWER: goodnight miss paine
PREDICTED ANSWER: goodnight paine paine 

row 7
QUESTION: you look better
REAL ANSWER: just better
PREDICTED ANSWER: i think 

row 8
QUESTION: you take japanese
REAL ANSWER: oh yes
PREDICTED ANSWER: oh oh 

row 9
QUESTION: i could of easy
REAL ANSWER: yeah i guess so
PREDICTED ANSWER: yeah yeah 

row 10
QUESTION: something to drink dr bright
REAL ANSWER: no thank you
PREDICTED ANSWER: no when you are 

row 11
QUESTION: uh youyou wanna lift
REAL ANSWER: oh whyuh yyyou g

In [16]:
batch_x, seq_x = pad_sentence_batch(X_test[:batch_size], PAD)
batch_y, seq_y = pad_sentence_batch(Y_test[:batch_size], PAD)
predicted = sess.run(model.predicting_ids, feed_dict={model.X:batch_x,model.X_seq_len:seq_x})

In [17]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: we landing
REAL ANSWER: something no dead
PREDICTED ANSWER: come cost on on it 

row 2
QUESTION: take a long view
REAL ANSWER: what is that bad pour
PREDICTED ANSWER: thank you you 

row 3
QUESTION: anything at the airport
REAL ANSWER: meeting mom am searched say
PREDICTED ANSWER: nothing been been 

row 4
QUESTION: this is magnificent
REAL ANSWER: bugs favor
PREDICTED ANSWER: i have 

row 5
QUESTION: war's over pal
REAL ANSWER: healthy donny that sleep
PREDICTED ANSWER: he is a houseafire 

row 6
QUESTION: uhdid not we udou this
REAL ANSWER: some taking a lover
PREDICTED ANSWER: i do not know 

row 7
QUESTION: oh migod
REAL ANSWER: over here greaser me
PREDICTED ANSWER: what is you 

row 8
QUESTION: where is my pill
REAL ANSWER: that olive often erik
PREDICTED ANSWER: what is it 

row 9
QUESTION: where is he
REAL ANSWER: cannot for arrangement
PREDICTED ANSWER: he is galleria 

row 10
QUESTION: captain you wanna take'er up
REAL ANSWER: mmmm thy academy how right
PREDIC